# 무신사 스크래이핑 
- [ ] 상품번호
- [ ] 브랜드 
- [ ] 제품명 
- [ ] 가격
- [ ] 제품 만족도 
- [ ] 썸네일 사진 - 001001-상품번호-thumb.jpg
- [ ] 제품 상세 사진 - 001001-상품번호-detail-(n).jpg


In [11]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, urlretrieve # urlretrieve : 이미지 다운

url = 'https://store.musinsa.com/app/items/lists/001001'

In [12]:
page = urlopen(url)
soup = BeautifulSoup(page, 'html.parser')

## 스크래이핑 전략 세우기
- 내가 스크래이핑 하고자 하는 엘리먼트를 포함하는 엘리먼트의 ID를 찾는다. 
- 최대한 ID값을 활용하라 .

In [13]:
li_box = soup.find('ul', id='searchList').find_all('li', class_='li_box')
len(li_box)

90

## 전체를 한번에 수집하는 것이 아닌, 하나의 아이템만 먼저 스크래이핑 하자 

In [15]:
brand = li_box[0].find('p', class_='item_title').text
product = li_box[0].find('p', class_='list_info').text
price = li_box[0].find('p', class_='price').text

In [16]:
brand, product, price

('그레이버',
 '\n\n\t\t\t\t[UNISEX]도트보드 자수 화이트클립 반팔티_2color\n',
 '\n\t\t\t\t\t\t\t\t42,000원\n\t\t\t')

In [20]:
# 상품번호 가져오기 
product_num = li_box[0].find('div', class_='list_img').find('a')['href']
product_num = product_num.split('/')[-2]
product_num

'1517122'

# 썸네일 저장하기
- path : ./musinsa/thumb

- 반복되는 문자열은 포매팅 기법을 적절하게 사용 

- 이미지 이름이 제품번호, 인덱스를 제외하고 다 비슷 

In [21]:
thumb_path = './musinsa/thumb/{}'

In [25]:
# 썸네일 url 가지고 오기 
thumb_url = li_box[0].find('div', class_='list_img').find('img')['data-original']
thumb_url

'//image.msscdn.net/images/goods_img/20200716/1517122/1517122_1_125.jpg'

In [27]:
urlretrieve('https:'+thumb_url, thumb_path.format('001001-{}-thumb.jpg').format(product_num))


('./musinsa/thumb/001001-1517122-thumb.jpg',
 <http.client.HTTPMessage at 0x114341e90>)

# 상세 페이지 url

In [28]:
detail_url = li_box[0].find('div', class_='list_img').find('a')['href']
detail_url

'/app/product/detail/1517122/0'

In [29]:
from urllib.parse import urljoin

base_url = 'https://store.musinsa.com/'

full_url = urljoin(base_url, detail_url)
full_url

'https://store.musinsa.com/app/product/detail/1517122/0'

# 후기 가지고 오기

In [31]:
detail_page = urlopen(full_url)
detail_soup = BeautifulSoup(detail_page, 'html.parser')

In [33]:
review_msg = detail_soup.find('div', class_='estimate-stats').text.strip()
review_msg

'등록된 후기가 없습니다.'

# 상세 이미지 가져오기 
- 규칙 확인 결과 : thum_[li의 인덱스]로 계속 늘어가는 것을 확인

In [34]:
img_li_list = detail_soup.find('ul', class_='product_thumb').find_all('li')
len(img_li_list)

1

In [36]:
img_id_format = 'thum_{}'

In [52]:
thumb_img_name  = '001001-{}-thumb.jpg'
detail_img_name = '001001-{}-detail-{}.jpg'

detail_img_path = './musinsa/detail/{}'
for idx, li in enumerate(img_li_list):
    img = li.find('img', id=img_id_format.format(idx))
    img_url = img['src']
    img_url_split = img_url.split('/')
    img_original_name = img_url_split[-1]
    
    img_url_complete = 'https:'+'/'.join(img_url_split[:-1])+'/'+img_original_name[:-6]+'500.jpg'
    urlretrieve(img_url_complete, detail_img_path.format('이미지.jpg'))

# 1. 한 페이지 당의 전체 상품 크롤링 

In [65]:
import pandas as pd
import tqdm

base_url = 'https://store.musinsa.com'
list_url = '/app/items/lists/001001'

url = 'https://store.musinsa.com/app/items/lists/001001'
page = urlopen(url)
soup = BeautifulSoup(page, 'html.parser')

thumb_img_name  = '001001-{}-thumb.jpg'
detail_img_name = '001001-{}-detail-{}.jpg'

thumb_path = './musinsa/thumb/{}'
detail_path = './musinsa/detail/{}'

li_box = soup.find('ul', id='searchList').find_all('li', class_='li_box')


product_num_list = []
brand_list = []
product_name_list = []
price_list = []
review_list = []
thumb_list = []
detail_list = []



for li in tqdm.tqdm_notebook(li_box):
    brand = li_box[0].find('p', class_='item_title').text.strip()
    product_name = li_box[0].find('p', class_='list_info').text.strip()
    price = li_box[0].find('p', class_='price').text.strip()
    product_num = li.find('div', class_='list_img').find('a')['href'].split('/')[-2]
    thumbnail = thumb_img_name.format(product_num)
    
    product_num_list.append(product_num)
    brand_list.append(brand)
    price_list.append(price)
    product_name_list.append(product_name)
    thumb_list.append(thumbnail)
    
    # 이미지 다운로드
    thumb_url = li.find('div', class_='list_img').find('a').find('img')['data-original']
    urlretrieve('https:'+ thumb_url, thumb_path.format(thumbnail))
    
    detail_url = li.find('div', class_='list_img').find('a')['href']
    
    # 디테일 페이지 크롤링 시작 
    full_url = urljoin(base_url, detail_url)
    detail_page = urlopen(full_url)
    detail_soup = BeautifulSoup(detail_page, 'html.parser')
    
    # 디테일 페이지에서 후기 가져오기
    review_msg = detail_soup.find('div', class_='estimate-stats').text.strip()
    review_list.append(review_msg)
    
    # 실제 이미지 목록 가져오기
    img_li_list = detail_soup.find('ul', class_='product_thumb').find_all('li')
    
    detail_img_list = []
    for idx, li in enumerate(img_li_list):
        img = li.find('img', id=img_id_format.format(idx))
        img_url = img['src']
        img_url_split = img_url.split('/')
        img_original_name = img_url_split[-1]
        
        img_url_complete = 'https:'+'/'.join(img_url_split[:-1])+'/'+img_original_name[:-6]+'500.jpg'
        detail_name = detail_img_name.format(product_num, idx)
        detail_img_list.append(detail_name)
        urlretrieve(img_url_complete, detail_img_path.format(detail_name))
        
    detail_list.append('#'.join(detail_img_list))
    


/Users/son/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [66]:
df_musinsa = pd.DataFrame({
    '제품번호':product_num_list,
    '브랜드':brand_list,
    '제품명':product_name_list,
    '가격':price_list,
    '썸네일':thumb_list,
    '리뷰':review_list,
    '상세사진':detail_list
})

df_musinsa.to_csv('./data/musinsa.csv', encoding='utf-8')
    

In [67]:
df_musinsa.head()

,제품번호,브랜드,제품명,가격,썸네일,리뷰,상세사진
0,787664,[클리어런스],DOX-607 SIMPLE POCKET 1/2 T - BLACK,"24,000원\t\t\t\t6,900원",001001-787664-thumb.jpg,94.9% 만족,001001-787664-detail-0.jpg
1,787665,[클리어런스],DOX-607 SIMPLE POCKET 1/2 T - BLACK,"24,000원\t\t\t\t6,900원",001001-787665-thumb.jpg,94.9% 만족,001001-787665-detail-0.jpg
2,1517408,[클리어런스],DOX-607 SIMPLE POCKET 1/2 T - BLACK,"24,000원\t\t\t\t6,900원",001001-1517408-thumb.jpg,등록된 후기가 없습니다.,001001-1517408-detail-0.jpg#001001-1517408-det...
3,1517392,[클리어런스],DOX-607 SIMPLE POCKET 1/2 T - BLACK,"24,000원\t\t\t\t6,900원",001001-1517392-thumb.jpg,등록된 후기가 없습니다.,001001-1517392-detail-0.jpg#001001-1517392-det...
4,1508488,[클리어런스],DOX-607 SIMPLE POCKET 1/2 T - BLACK,"24,000원\t\t\t\t6,900원",001001-1508488-thumb.jpg,등록된 후기가 없습니다.,001001-1508488-detail-0.jpg#001001-1508488-det...
